In [1]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리_2019.csv')
data_raw

,Unnamed: 0,만나이,성별,주관적 건강수준,격렬한 신체활동 일수,격렬한 신체활동 시간(시),격렬한 신체활동 시간(분),중등도 신체활동 일수,중등도 신체활동 시간(시),중등도 신체활동 시간(분),...,우울감 경험 여부,행복감 지수,사회환경_동네 자연환경,사회환경_동네 생활환경,가구원수,가구소득_년월,가구소득_년금액,가구소득_월금액,조건부가구소득,자치구
0,0,61,2,4,1,5,0,3,1,0,...,2,5,2,1,2,9.0,99999.0,99999.0,4.0,마포구
1,1,66,1,3,1,5,0,0,88,88,...,2,8,2,1,2,9.0,99999.0,99999.0,4.0,마포구
2,2,89,2,5,0,88,88,0,88,88,...,2,77,9,9,1,2.0,88888.0,60.0,88.0,마포구
3,3,27,1,4,1,0,10,1,0,10,...,2,8,1,1,1,2.0,88888.0,200.0,88.0,마포구
4,4,42,2,3,2,0,30,2,0,10,...,2,6,2,2,3,2.0,88888.0,200.0,88.0,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22925,22925,62,2,3,0,88,88,7,1,0,...,2,4,9,1,2,2.0,88888.0,150.0,88.0,금천구
22926,22926,65,1,4,7,2,0,7,2,0,...,2,10,2,2,2,2.0,88888.0,150.0,88.0,금천구
22927,22927,24,1,2,0,88,88,0,88,88,...,2,7,2,1,3,2.0,88888.0,300.0,88.0,금천구
22928,22928,56,2,3,0,88,88,0,88,88,...,2,7,1,1,3,2.0,88888.0,300.0,88.0,금천구


In [2]:
df = data_raw[['만나이','성별','가구원수','가구소득_년월','가구소득_년금액','가구소득_월금액','자치구']]

In [3]:
df_hi1=df[df['가구소득_년금액'] != 99999]
df_hi2=df_hi1[df_hi1['가구소득_년금액'] != 77777]
df_hi3=df_hi2[df_hi2['가구소득_월금액'] != 99999]
df_hi4=df_hi3[df_hi3['가구소득_월금액'] != 77777]
df_hi5=df_hi4[df_hi4['가구소득_년월'] != 9]
df_hi6=df_hi5[df_hi5['가구소득_년월'] != 7]

In [4]:
def transincome(df_income) :
    if df_income['가구소득_년월'] == 1: # 1은 년 
        return df_income['가구소득_년금액'] / df_income['가구원수']
    elif df_income['가구소득_년월'] == 2: # 2는 월
        return df_income['가구소득_월금액']*12  / df_income['가구원수']
    else :
        return 0

In [5]:
df_income = df_hi6.copy() # 오류제거
df_income["소득 연봉 환산"] = df_income.apply(transincome, axis =1)
df_income = df_income.drop(['가구소득_년월','가구소득_년금액','가구소득_월금액','가구원수'],axis=1)
df_income

,만나이,성별,자치구,소득 연봉 환산
2,89,2,마포구,720.0
3,27,1,마포구,2400.0
4,42,2,마포구,800.0
5,74,1,마포구,800.0
6,71,2,마포구,800.0
...,...,...,...,...
22925,62,2,금천구,900.0
22926,65,1,금천구,900.0
22927,24,1,금천구,1200.0
22928,56,2,금천구,1200.0


In [6]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [9]:
list_df = []
for value in list_region:
    temp = df_income['자치구'] == value
    list_df.append(df_income[temp])
list_df[0] = df_income

In [14]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_hi = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] <= int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            temp3 = temp2["소득 연봉 환산"].value_counts().to_dict()
            total_sum = 0
            for m,n in temp3.items():
                total_sum += m*n
            value_hi.append(round(total_sum/len(temp2['소득 연봉 환산']),1))
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [16]:
df_final = pd.DataFrame({'1인당 연소득':value_hi, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final

1인당 연소득
지역  연령별   성별         
서울시 계     계    2029.6
          남    2066.9
          여    2000.3
    19~44 계    2235.7
          남    2270.1
...               ...
강남구 65~74 남    2657.1
          여    2112.3
    75 이상 계    1957.8
          남    1971.8
          여    1949.7

[390 rows x 1 columns]

In [17]:
df_final.to_csv("../raw/지역사회건강조사 (전처리된 데이터)_2019/지역사회건강조사 (1인당 연소득)_2019.csv", mode='w',encoding = 'utf-8-sig')